In [276]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

RANDOM_STATE = 42

In [277]:
# Load the dataset using pandas
df = pd.read_csv("data.csv")

print(df.head())
df = df.drop('I', axis=1)

## Removing our target variable

selected_features = ["qPA", "Pulse", "BreathFreq"]
X = df[selected_features].values
y_gravity = df["Gravity"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


print(f"Temperature Max, Min post normalization: {np.max(X_scaled[0]):0.2f}, {np.min(X_scaled[0]):0.2f}")

   I     P_sist     P_dist       qPA       Pulse  BreathFreq    Gravity  Class
0  1  13.592433  12.220855  8.416754   75.921057   21.635259  40.000000      2
1  2  15.775386  13.586879  8.725890   63.813564   19.718734  41.530427      2
2  3   3.649369   1.904802  0.000000  197.210213   19.045471  52.730745      3
3  4  17.264362  13.700638  8.733333  143.636181   17.621141  34.679911      2
4  5  12.705183   9.485389  1.747626   82.636672   12.209535  69.375882      3
Temperature Max, Min post normalization: 1.72, -0.44


In [278]:
cat_variables = ['Class']

# This will replace the columns with the one-hot encoded ones and keep the columns outside 'columns' argument as it is.
df = pd.get_dummies(data = df,
                         prefix = "class",
                         columns = cat_variables)

print(df.head())

      P_sist     P_dist       qPA       Pulse  BreathFreq    Gravity  class_1  \
0  13.592433  12.220855  8.416754   75.921057   21.635259  40.000000    False   
1  15.775386  13.586879  8.725890   63.813564   19.718734  41.530427    False   
2   3.649369   1.904802  0.000000  197.210213   19.045471  52.730745    False   
3  17.264362  13.700638  8.733333  143.636181   17.621141  34.679911    False   
4  12.705183   9.485389  1.747626   82.636672   12.209535  69.375882    False   

   class_2  class_3  class_4  
0     True    False    False  
1     True    False    False  
2    False     True    False  
3     True    False    False  
4    False     True    False  


In [307]:
# Define X (features), y_class e y_gravity
y_class = df[["class_1", "class_2", "class_3", "class_4"]].values
gravity = df["Gravity"].values

# Divide em treino e teste
X_train, X_test, y_train, y_test, y_class_train, y_class_test, gravity_train, gravity_test = train_test_split(X, y_gravity, y_class, gravity, train_size = 0.7, random_state = RANDOM_STATE)

print(f'train samples: {len(X_train)}\ntest samples: {len(X_test)}')

train samples: 1050
test samples: 450


In [282]:
print(X_train.shape, y_class_train.shape)
print(X.shape[1])

(1050, 3) (1050, 4)
3


In [283]:
tf.random.set_seed(1234)  # applied to achieve consistent results

model = tf.keras.Sequential([
    tf.keras.Input(X.shape[1]),
    Dense(32, activation='relu', name = 'layer1'),
    Dense(16, activation='relu', name = 'layer2'),
    Dense(8, activation='relu', name = 'layer3'),
    Dense(1, activation='linear', name = 'output')  # Output for regression
])


In [284]:
model.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 32)                128       
                                                                 
 layer2 (Dense)              (None, 16)                528       
                                                                 
 layer3 (Dense)              (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [285]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)
early_stop = EarlyStopping(
    monitor='val_loss',        # what metric to monitor (can also use 'val_mae')
    patience=10,               # how many epochs to wait before stopping
    restore_best_weights=True # keep the best model, not the last one
)
model.fit(
    X_train, y_train,
    epochs=4000,
    callbacks=[early_stop],    # here's the EarlyStopping callback
    verbose=1
)

Epoch 1/4000
33/33 [==============================] - 0s 1ms/step - loss: 2235.6667 - mae: 43.9749
Epoch 2/4000
33/33 [==============================] - 0s 1ms/step - loss: 1344.4774 - mae: 32.1761
Epoch 3/4000
33/33 [==============================] - 0s 2ms/step - loss: 880.7625 - mae: 25.0060
Epoch 4/4000
33/33 [==============================] - 0s 1ms/step - loss: 737.8790 - mae: 23.2173
Epoch 5/4000
33/33 [==============================] - 0s 2ms/step - loss: 654.6829 - mae: 21.8128
Epoch 6/4000
33/33 [==============================] - 0s 1ms/step - loss: 527.0120 - mae: 19.3894
Epoch 7/4000
33/33 [==============================] - 0s 1ms/step - loss: 366.1396 - mae: 15.6617
Epoch 8/4000
33/33 [==============================] - 0s 1ms/step - loss: 354.9765 - mae: 15.1181
Epoch 9/4000
33/33 [==============================] - 0s 2ms/step - loss: 338.6779 - mae: 14.6953
Epoch 10/4000
33/33 [==============================] - 0s 2ms/step - loss: 333.4614 - mae: 14.5907
Epoch 11/4000
33/

In [286]:
def regression_accuracy(model, X_test, y_test, tolerance=3.7):
    """
    Calculates the percentage of predictions within a tolerance of the true value.
    Args:
        model: Trained Keras model
        X_test: Test features
        y_test: True values
        tolerance: Acceptable error (absolute difference)
    Returns:
        accuracy: Percentage of predictions within tolerance
    """
    y_pred = model.predict(X_test).flatten()
    correct = np.abs(y_pred - y_test) <= tolerance
    accuracy = np.mean(correct)
    print(f"Regression accuracy (within ±{tolerance}): {accuracy*100:.2f}%")
    return accuracy

# Example usage:
regression_accuracy(model, X_test, y_test)

15/15 [==============================] - 0s 2ms/step
Regression accuracy (within ±3.7): 84.67%


0.8466666666666667

In [287]:
loss, accuracy = model.evaluate(X_test, y_test)

15/15 [==============================] - 0s 2ms/step - loss: 9.1917 - mae: 2.0368


In [ ]:
y_pred = model.predict(X_test)  # shape: (num_samples, 1)

# Build the classifier
clf_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),  # Input is y_pred from regression
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

clf_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the classifier
print(f"y_pred shape: {y_pred.shape}, y_class_test shape: {y_class_test.shape}")
clf_model.fit(gravity_train, y_class_train, epochs=1000, verbose=1, callbacks=[early_stop])

 1/15 [=>............................] - ETA: 0s

15/15 [==============================] - 0s 2ms/step
y_pred shape: (450, 1), y_class_test shape: (450, 4)
Epoch 1/1000
33/33 [==============================] - 0s 2ms/step - loss: 4.4149 - accuracy: 0.1838
Epoch 2/1000
33/33 [==============================] - 0s 1ms/step - loss: 1.0822 - accuracy: 0.5600
Epoch 3/1000
33/33 [==============================] - 0s 1ms/step - loss: 1.0292 - accuracy: 0.5533
Epoch 4/1000
33/33 [==============================] - 0s 1ms/step - loss: 1.0161 - accuracy: 0.5657
Epoch 5/1000
33/33 [==============================] - 0s 1ms/step - loss: 0.9966 - accuracy: 0.6705
Epoch 6/1000
33/33 [==============================] - 0s 2ms/step - loss: 1.0005 - accuracy: 0.5752
Epoch 7/1000
33/33 [==============================] - 0s 2ms/step - loss: 0.9797 - accuracy: 0.5962
Epoch 8/1000
33/33 [==============================] - 0s 2ms/step - loss: 0.9664 - accuracy: 0.6152
Epoch 9/1000
33/33 [==============================] - 0s 3ms/step - loss: 0.9572 - accuracy: 0

In [306]:
y_class_pred = np.argmax(clf_model.predict(y_pred), axis=1)
print("Predicted classes:", y_class_pred)

15/15 [==============================] - 0s 972us/step
Predicted classes: [1 1 1 1 2 1 2 2 1 1 1 2 1 1 0 2 1 1 0 1 0 1 1 0 2 1 1 1 2 1 1 1 1 1 1 2 1
 1 1 1 1 2 1 1 2 0 1 1 0 1 1 1 1 1 1 1 0 2 1 2 0 0 1 1 1 2 1 1 2 1 1 1 1 1
 2 1 2 1 0 1 1 2 2 2 2 2 1 1 2 1 1 2 1 1 1 1 2 1 1 1 2 1 1 2 0 1 2 2 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 1 0 1 2 1 2 1 1 1 2 1 1 0 1 1 1 0 0 1 1 1 2 1 2 2 1
 1 2 1 2 2 1 0 2 1 2 1 1 1 1 1 1 1 2 1 2 1 2 1 0 1 1 1 1 1 1 2 0 0 1 1 1 1
 0 1 2 0 1 0 1 2 1 2 1 1 1 2 0 1 2 0 1 1 0 1 2 2 1 2 2 1 1 1 2 1 2 0 1 1 2
 1 1 2 1 1 1 1 2 2 1 2 1 1 1 1 1 1 1 0 1 1 2 1 1 0 1 1 1 1 2 2 2 1 1 1 1 1
 1 2 1 1 2 1 0 2 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 0 1 1
 1 2 1 1 1 2 1 1 1 1 2 1 1 2 0 2 2 1 1 1 2 1 1 1 1 1 2 1 2 2 0 1 1 2 0 1 1
 1 1 2 1 1 1 2 2 2 1 2 1 1 1 2 1 1 1 1 2 2 1 2 1 2 1 1 1 1 2 1 1 1 1 0 0 1
 1 1 1 2 1 1 2 1 1 1 1 0 1 1 2 2 2 1 2 1 0 2 0 1 1 1 1 2 1 1 2 1 1 0 1 2 1
 1 2 1 1 0 1 0 1 2 1 2 2 1 1 0 0 2 1 1 2 2 1 0 1 0 2 0 1 1 1 1 1 2 1 1 1 1
 0 2 1 1 1 2]


In [301]:
print("True classes:", np.argmax(y_class_test, axis=1))

True classes: [0 0 1 1 1 1 2 2 1 1 1 3 1 1 0 2 1 2 0 1 0 1 1 0 2 1 0 1 2 1 1 1 1 1 2 2 1
 1 1 1 1 3 1 1 2 0 1 1 1 1 2 1 1 0 1 0 0 2 1 1 0 0 1 1 1 2 1 1 2 1 1 2 1 1
 2 1 2 0 0 1 1 2 2 2 2 2 1 0 2 1 1 2 1 1 1 1 2 1 1 1 2 0 1 2 0 1 2 2 1 2 1
 1 0 1 1 1 1 1 0 0 0 1 0 0 1 2 1 3 0 1 1 2 1 1 0 2 1 2 0 0 1 1 1 2 2 2 2 0
 1 2 1 2 2 1 0 2 1 2 1 1 1 2 2 1 1 2 1 2 1 2 1 0 1 1 1 1 1 1 2 0 0 1 1 1 1
 0 1 2 0 1 0 1 3 1 2 2 1 1 2 0 1 2 0 1 1 0 1 3 2 1 2 3 1 1 1 2 1 2 0 1 1 2
 1 1 2 1 1 1 1 2 2 1 2 1 1 1 1 1 1 1 0 1 1 2 1 1 0 1 1 1 1 2 2 2 1 1 0 1 2
 1 2 1 1 2 1 0 2 1 2 2 1 1 1 1 1 2 1 0 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 0 1 1
 1 2 1 1 1 2 1 1 1 1 2 1 1 2 0 2 2 1 1 1 2 1 1 1 0 1 3 1 2 2 0 1 1 2 0 1 0
 1 1 2 1 1 1 2 2 2 1 2 1 0 1 2 1 1 1 1 2 2 1 2 1 2 1 1 1 1 2 1 1 2 1 0 0 1
 1 1 1 2 1 1 3 1 1 1 1 1 2 1 2 2 3 1 2 1 0 2 0 1 1 1 1 1 1 1 2 1 1 1 1 2 1
 1 1 0 1 0 1 0 1 2 1 2 2 1 1 0 0 2 1 1 2 2 2 0 1 0 2 0 1 1 1 1 1 2 0 1 1 1
 0 2 1 2 1 2]
